In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import pandas as pd

In [2]:
text = open('quote-text.txt', 'r').read()
print(f'length of text: {len(text)} characters')

length of text: 1071892 characters


In [3]:
print(text[:250])



I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.

You've gotta dance like there's nobody watch


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')
vocab

75 unique characters


['\n',
 ' ',
 '!',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [6]:
#Before training, you need to convert the strings to a numerical representation.
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding = 'UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary = list(vocab), mask_token = None)
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[50, 51, 52, 53, 54, 55, 56], [73, 74, 75]]>

In [8]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [9]:
tf.strings.reduce_join(chars,axis = -1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [10]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis = -1)

In [11]:
all_ids = ids_from_chars(tf.strings.unicode_split(text,'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [12]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))






I
'
m
 
s
e
l
f


In [13]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
sequences = ids_dataset.batch(seq_length+1, drop_remainder = True)
for seq in sequences.take(1):
  print(chars_from_ids(seq))


tf.Tensor(
[b'\n' b'\n' b'I' b"'" b'm' b' ' b's' b'e' b'l' b'f' b'i' b's' b'h' b','
 b' ' b'i' b'm' b'p' b'a' b't' b'i' b'e' b'n' b't' b' ' b'a' b'n' b'd'
 b' ' b'a' b' ' b'l' b'i' b't' b't' b'l' b'e' b' ' b'i' b'n' b's' b'e'
 b'c' b'u' b'r' b'e' b'.' b' ' b'I' b' ' b'm' b'a' b'k' b'e' b' ' b'm'
 b'i' b's' b't' b'a' b'k' b'e' b's' b',' b' ' b'I' b' ' b'a' b'm' b' '
 b'o' b'u' b't' b' ' b'o' b'f' b' ' b'c' b'o' b'n' b't' b'r' b'o' b'l'
 b' ' b'a' b'n' b'd' b' ' b'a' b't' b' ' b't' b'i' b'm' b'e' b's' b' '
 b'h' b'a' b'r'], shape=(101,), dtype=string)


In [14]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text


In [15]:
split_input_target(list('tensorflow'))

(['t', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [16]:
dataset = sequences.map(split_input_target)
dataset

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [17]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"\n\nI'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times ha"
Target: b"\nI'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times har"


In [18]:
BATCH_SIZE = 64
# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements)
BUFFER_SIZE = 10000

dataset = (dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [19]:
#Length of the vocabulary in chars
vocab_size = len(vocab)
#The embedding dimension
embedding_dim = 256
#Number of RNN Units
rnn_units = 1024

In [20]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x
model = MyModel (
        #be sure the vocabulary size matches the 'StringLookup' layers.
        vocab_size = len(ids_from_chars.get_vocabulary()), embedding_dim = embedding_dim, rnn_units = rnn_units
)


In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 76) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  19456     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  77900     
                                                                 
Total params: 4,035,660
Trainable params: 4,035,660
Non-trainable params: 0
_________________________________________________________________


In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices


array([71,  6, 69, 37, 14, 59, 45, 42, 28, 17, 31, 40, 11,  6, 15, 10, 32,
       29,  1,  7, 52, 53, 67, 55, 32, 46, 68, 22,  4, 60, 58, 35, 41, 60,
       67, 14, 69,  5,  3, 60, 56, 27, 49,  2,  8,  0, 70,  5, 24, 12, 68,
       49, 35, 29, 29, 37, 13, 59, 32, 53, 72, 20, 75,  0, 62, 74, 22, 47,
       27,  7, 58, 48,  7, 37, 42, 20, 65, 28, 62, 58, 47, 45,  4, 49, 39,
       19, 11, 13, 54, 37, 13, 74, 41, 74, 17, 37, 12, 34, 75,  4])

In [24]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"u live it will be in your armswithout leaving mine.\n\nYours is the light by which my spirit's born: -"

Next Char Predictions:
 b"v(tN3jVSE6HQ0(4.IF\n)cdrfIWs;&kiLRkr3t'!kgDZ ,[UNK]u'A1sZLFFN2jIdw9z[UNK]my;XD)iY)NS9pEmiXV&ZP802eN2yRy6N1Kz&"


In [25]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 76)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.3310246


In [33]:
tf.exp(mean_loss).numpy()

76.02215

In [34]:
model.compile(optimizer='adam', loss=loss)

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [36]:
EPOCHS = 10

In [37]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
165/165 [==============================] - 28s 137ms/step - loss: 1.3925
Epoch 2/10
165/165 [==============================] - 24s 135ms/step - loss: 1.3150
Epoch 3/10
165/165 [==============================] - 24s 136ms/step - loss: 1.2578
Epoch 4/10
165/165 [==============================] - 24s 135ms/step - loss: 1.2078
Epoch 5/10
165/165 [==============================] - 24s 135ms/step - loss: 1.1596
Epoch 6/10
165/165 [==============================] - 24s 135ms/step - loss: 1.1117
Epoch 7/10
165/165 [==============================] - 24s 136ms/step - loss: 1.0619
Epoch 8/10
165/165 [==============================] - 24s 136ms/step - loss: 1.0092
Epoch 9/10
165/165 [==============================] - 24s 136ms/step - loss: 0.9542
Epoch 10/10
165/165 [==============================] - 24s 136ms/step - loss: 0.8965


In [38]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states
    

In [39]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [45]:
start = time.time()
states = None
next_char = tf.constant(["Once upon a time"])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Once upon a time would ever make my mind up at you, and still see his souls the singular pleasure champing wings as they also believe: Denuer, there is a word surprises ang right. When all let stories, the taste of your soul lasts eye and feel here. A long time, times fragile, earning by one deserve brave. A woman becomes the things we did their minds in the sepanding of a girl will become negative 'dill and be loved by ladies, that is what it works.

I'll be sigzon's story eternal such is an agreesboxt door.

He looked so we're anl ordinary passionate - Dreaming, the kind of person with unrealized.

If there crosses sy long enough, attracted ones and show, the door will be the gates to climb a thousand madic don for the purpose of just because her no oceancies normally face and neither hand it to him, then future never quite perfection.

I never want to keep going. If it was not the way you think you have to do something, we don't, shinking. And who is borrowed. Maybe even foone,There